# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import holoviews as hv
import geoviews as gv
import hvplot.pandas
import requests

import matplotlib.pyplot as plt

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../WeatherPy/city_weather_data.csv")

# Display sample data
city_data_df.head()



,City,Lat,lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,san antonio de pale,-1.4014,5.6325,73.31,75.0,93.0,11.52,GQ,1.722981e+09
1,grytviken,-54.2811,-36.5092,16.30,72.0,26.0,7.87,GS,1.722980e+09
2,bamboo flat,11.7000,92.7167,78.89,89.0,100.0,2.30,IN,1.722981e+09
3,port alfred,-33.5906,26.8910,57.65,75.0,0.0,8.30,ZA,1.722981e+09
4,west island,-12.1568,96.8225,80.58,74.0,100.0,13.80,CC,1.722981e+09


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:


%%capture --no-display

humidity = city_data_df["Humidity"].astype(int)

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values

#A max temperature lower than 27 degrees but higher than 21
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 85) & (city_data_df["Max Temp"] > 70)]

city_data_df

# # #Wind speed less than 4.5 m/s
city_data_df = city_data_df.loc[(city_data_df["Wind Speed"] < 4.5)]

city_data_df

# # #Zero cloudiness
city_data_df = city_data_df.loc[(city_data_df["Cloudiness"] == 0)]

city_data_df

# # # Drop results with null values
city_data_df = city_data_df.dropna()

# # # Display sample data
city_data_df

,City,Lat,lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
26,colonia,50.9333,6.9500,74.97,75.0,0.0,1.14,DE,1.722981e+09
29,aguilas,37.4063,-1.5829,81.50,62.0,0.0,1.86,ES,1.722981e+09
75,yigo village,13.5361,144.8886,79.92,88.0,0.0,0.00,GU,1.722981e+09
169,saipan,15.1355,145.7010,81.30,89.0,0.0,3.44,MP,1.722981e+09
286,sala,34.0389,-6.8166,73.63,94.0,0.0,3.44,MA,1.722981e+09
317,darwin,-12.4611,130.8418,73.38,83.0,0.0,3.44,AU,1.722981e+09
404,viradouro,-20.8731,-48.2969,73.87,32.0,0.0,4.43,BR,1.722981e+09
419,carboneras,37.0015,-1.9213,82.31,67.0,0.0,4.07,ES,1.722981e+09
467,sho'rchi,38.0141,67.7899,83.28,19.0,0.0,1.07,UZ,1.722981e+09
473,protaras,35.0125,34.0583,79.00,89.0,0.0,1.01,CY,1.722981e+09


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'lng','Lat', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,lng,Lat,Humidity,Hotel Name
26,colonia,DE,6.9500,50.9333,75.0,
29,aguilas,ES,-1.5829,37.4063,62.0,
75,yigo village,GU,144.8886,13.5361,88.0,
169,saipan,MP,145.7010,15.1355,89.0,
286,sala,MA,-6.8166,34.0389,94.0,
317,darwin,AU,130.8418,-12.4611,83.0,
404,viradouro,BR,-48.2969,-20.8731,32.0,
419,carboneras,ES,-1.9213,37.0015,67.0,
467,sho'rchi,UZ,67.7899,38.0141,19.0,
473,protaras,CY,34.0583,35.0125,89.0,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
        "categories": "accomodation.hotel",
        "filter":'',
        "bias": '',
        "apiKey": geoapify_key,
        "limit": 1 
        }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lon = row['lng']
    lat = row['Lat']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
colonia - nearest hotel: No hotel found
aguilas - nearest hotel: No hotel found
yigo village - nearest hotel: No hotel found
saipan - nearest hotel: No hotel found
sala - nearest hotel: No hotel found
darwin - nearest hotel: No hotel found
viradouro - nearest hotel: No hotel found
carboneras - nearest hotel: No hotel found
sho'rchi - nearest hotel: No hotel found
protaras - nearest hotel: No hotel found
agat village - nearest hotel: No hotel found
morro agudo - nearest hotel: No hotel found
kas - nearest hotel: No hotel found
umatilla - nearest hotel: No hotel found


,City,Country,lng,Lat,Humidity,Hotel Name
26,colonia,DE,6.9500,50.9333,75.0,No hotel found
29,aguilas,ES,-1.5829,37.4063,62.0,No hotel found
75,yigo village,GU,144.8886,13.5361,88.0,No hotel found
169,saipan,MP,145.7010,15.1355,89.0,No hotel found
286,sala,MA,-6.8166,34.0389,94.0,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)